In [ ]:
%pip install -U 'crewai[tools]'
%pip install -U crewai
%pip install -U requests
%pip install -U python-dotenv

In [ ]:
from crewai import Crew, Agent, Task
import os
from dotenv import load_dotenv
from crewai_tools import ScrapeWebsiteTool

In [ ]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or "<your_openai_key>"
SERPER_API_KEY = os.getenv("SERPER_API_KEY") or "<your_serper_key>"

In [ ]:
from crewai_tools import BaseTool
import requests
import json
import os
from dotenv import load_dotenv

load_dotenv()

class CustomSerperDevTool(BaseTool):
    name: str = "Custom Serper Dev Tool"
    description: str = "Search the internet for news."

    def _run(self, query: str) -> str:
        """
        Search the internet for news.
        """

        url = "https://google.serper.dev/news"

        payload = json.dumps({
            "q": query,
            "num": 20,
            "autocorrect": False,
            "tbs": "qdr:d"
        })

        headers = {
            'X-API-KEY': SERPER_API_KEY,
            'Content-Type': 'application/json'
        }

        response = requests.request("POST", url, headers=headers, data=payload)

        # Parse the JSON response
        response_data = response.json()

        # Extract only the 'news' property
        news_data = response_data.get('news', [])

        # Convert the news data back to a JSON string
        return json.dumps(news_data, indent=2)



In [ ]:
custom_serper_dev_tool = CustomSerperDevTool()
scrape_website_tool = ScrapeWebsiteTool()

class Agents:
    def researcher(self, topic):
        return Agent(
            role=f"{topic} Senior News Researcher",
            goal=f"Uncover latest news in {topic}",
            tools=[custom_serper_dev_tool, scrape_website_tool],
            backstory=f"You're a seasoned researcher with a knack for uncovering the latest developments in {topic}. Known for your ability to find the most relevant information and present it in a clear and concise manner.",
            verbose=True,
        )

    def reporting_analyst(self, topic):
        return Agent(
            role=f"{topic} News Reporting Analyst",
            goal=f"Create detailed reports based on {topic} news analysis and research findings",
            tools=[],
            backstory=f"You're a meticulous analyst with a keen eye for detail. You're known for your ability to turn complex data into clear and concise reports, making it easy for others to understand and act on the information you provide.",
            verbose=True,
        )

In [ ]:
from textwrap import dedent

class Tasks:
    def research_task(self, agent, topic):
        return Task(
            description=dedent(
                f"""
                    Search news about {topic}
                    """
            ),
            expected_output=dedent(
                f"""A list of news articles about {topic} with the title, url and snippet"""
            ),
            agent=agent,
        )

    def reporting_task(self, agent):
        return Task(
            description=dedent(
                f"""
                    Review the context you got.
                    Make sure the report is detailed and contains any and all relevant information.
                """
            ),
            expected_output=dedent(
                """A fully fledge reporting of the news articles.
                Formatted as markdown without '```'"""
            ),
            agent=agent,
        )

In [ ]:
from datetime import datetime

tasks = Tasks()
agents = Agents()

topic = "ai agents"

# Create Agents
researcher = agents.researcher(topic=topic)
reporting_analyst = agents.reporting_analyst(topic=topic)

# Define Tasks for each agent
research_task = tasks.research_task(agent=researcher, topic=topic, date=datetime.now().strftime("%Y-%m-%d"))
reporting_task = tasks.reporting_task(agent=reporting_analyst)

# Instantiate the crew with a sequential process
crew = Crew(
    agents=[researcher, reporting_analyst],
    tasks=[
        research_task,
        reporting_task,
    ],
)



In [ ]:
import asyncio

from crewai.flow.flow import Flow, listen, start, and_
from litellm import completion
from pydantic import BaseModel

class News(BaseModel):
    news: str = ""

class NewsFlow(Flow[News]):
    model = "gpt-4o-mini"
    model_4o = "gpt-4o"

    @start()
    def generate_news_topic(self):
        print("Starting flow")

        response = completion(
            model=self.model_4o,
            messages=[
                {
                    "role": "user",
                    "content": """Return a topic within the ai world that is trending.  
                    This should be 1 - 4 words.""",
                },
            ],
        )

        news_topic = response["choices"][0]["message"]["content"]

        print(f"News Topic: {news_topic}")

        return news_topic

    @listen(generate_news_topic)
    def generate_news(self, news_topic):
        print("Generating news with Crew")

        inputs = {
            'topic': news_topic
        }

        result = crew.kickoff(inputs=inputs)

        # get raw output then save to state
        output = result.raw
        self.state.news = output

        return output

    @listen(generate_news)
    def save_news(self, news):
        print("Saving news")
        
        # Create the news directory if it doesn't exist
        news_dir = os.path.join(os.path.dirname(__file__), "..", "..", "news")
        os.makedirs(news_dir, exist_ok=True)
        
        # Save the news in the news directory
        news_file_path = os.path.join(news_dir, "news.md")
        with open(news_file_path, "w") as f:
            f.write(news)
        
        print(f"News saved to: {news_file_path}")

    @listen(generate_news)
    def generate_best_news(self, input):
        print("Generating best news")
        
        response = completion(
            model=self.model,
            messages=[
                {
                    "role": "user",
                    "content": f"Choose the most important news from the following and return it: {input}",
                },
            ],
        )

        important_news = response["choices"][0]["message"]["content"]
        return important_news
    
    @listen(and_(generate_news_topic, generate_news, save_news, generate_best_news))
    def logger(self, result):
        print(f"Logger: {result}")
        print("*" * 100)
        print("News Complete!")

In [ ]:
async def main():
    flow = NewsFlow()
    flow.plot("my_flow_plot")

    await flow.kickoff()

asyncio.run(main())